In [1]:
import minsearch

In [2]:
import json

In [3]:
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)

In [7]:
documents=[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
    

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [14]:
index = minsearch.Index(
    text_fields=['question','text','section'],
                keyword_fields=['course']
)

In [12]:
q = 'the course has already started, can I still enroll?'

In [15]:
index.fit(documents)

In [21]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query = q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [31]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

### Now, add the LLM component

In [23]:
from openai import OpenAI

In [24]:
client = OpenAI()

In [25]:
q

'the course has already started, can I still enroll?'

In [26]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":q}]
)

In [27]:
response

ChatCompletion(id='chatcmpl-Bcgj3Pv4sRFSpw8oEiHCb8zN90L3R', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Enrollment policies vary by course and institution. It's best to contact the course administrator or admissions office directly to inquire about late enrollment. Some courses may allow late enrollment within a certain timeframe, while others may not. Be sure to check if there are any specific requirements or steps you need to follow to join the course after it has started.", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1748560401, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_a288987b44', usage=CompletionUsage(completion_tokens=66, prompt_tokens=18, total_tokens=84, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), pr

In [ ]:
# response.choices[0].message.content

### This is only using our knowledge base, need to connect to GPT

In [28]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION asked on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.  
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: {question}
CONTEXT: {context}
"""

In [32]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer:{doc['text']}\n\n"

In [33]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer:Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer:The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with t

In [34]:
prompt = prompt_template.format(question=q, context=context).strip()

In [35]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION asked on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.  
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: the course has already started, can I still enroll?
CONTEXT: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer:Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your fi

### Now ready for GPT

In [38]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":prompt}]
)

In [37]:
response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

### the next phase is to simplify with functions

In [50]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query = query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return results

In [43]:
def build_prompt(query,search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION asked on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.  
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: {question}
CONTEXT: {context}
""".strip()

    context = ""

    for doc in results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer:{doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [44]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role":"user","content":prompt}]
    )
    
    return response.choices[0].message.content    

In [45]:
query = 'the course already started, can I still enroll?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query,search_results)
    answer = llm(prompt)
    return answer

In [46]:
rag(query)

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [47]:
rag('how do I run kafka?')

'NONE'

In [51]:
rag('do we need to install docker?')


'NONE'